## Pages List (Buy)

In [27]:
import requests
import regex
from bs4 import BeautifulSoup

pages_list = []
for i in range(1, 839):
    pages_list.append('https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|'+ str(i) +'||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||')
    print('Paginator #' + str(i) + ": " + pages_list[i-1])

Paginator #1: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||
Paginator #2: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|2||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||
Paginator #3: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|3||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||
Paginator #4: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|4||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||
Paginator #5: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|5||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||
Paginator #6: https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|6||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500

## Properties Pages List (Buy)

In [28]:
paginator_url = 'https://www.fincaraiz.com.co/finca-raiz/venta/medellin/?ad=30|1||||1||8,9,3,4,22,2,5,7,19,23,21,18,20|||55|5500006||||||||||||||||1|||1||griddate%20desc||||-1||'
page = requests.get(paginator_url)
soup = BeautifulSoup(page.content, 'html.parser')
division = soup.find(id='divAdverts')
properties_pages_list = []
for i in range(0, 34):
    properties = division.find(id='rowIndex_' + str(i))
    property = ''
    if (i == 0):
        property = properties.find('li', class_='media')
        property = property.find('span').attrs.get('onclick')
    else:
        property = properties.find('li', class_='media').attrs.get('onclick')
    url_property = str(property).replace('javascript:window.location=', '')
    url_property = 'https://www.fincaraiz.com.co' + url_property.replace("'", "")
    print('Property #' + str(i) + ": " + str(url_property))
    #properties_pages_list.append(url_property)

Property #0: https://www.fincaraiz.com.co/palermo/medellin/proyecto-nuevo-det-2393445.aspx
Property #1: https://www.fincaraiz.com.co/casa-en-venta/medellin/poblado-det-4955241.aspx
Property #2: https://www.fincaraiz.com.co/apartamento-en-venta/medellin/ciudad_del_rio-det-4889748.aspx
Property #3: https://www.fincaraiz.com.co/tibet/medellin/proyecto-nuevo-det-4568479.aspx?itemid=4568489
Property #4: https://www.fincaraiz.com.co/apartamento-en-venta/medellin/poblado-det-4425496.aspx
Property #5: https://www.fincaraiz.com.co/apartamento-en-venta/medellin/calasanz-det-4924487.aspx
Property #6: https://www.fincaraiz.com.co/tibet/medellin/proyecto-nuevo-det-4568479.aspx?itemid=4568488
Property #7: https://www.fincaraiz.com.co/lote-en-venta/medellin/nororiente-det-4970078.aspx
Property #8: https://www.fincaraiz.com.co/senderos-de-calasanz/medellin/proyecto-nuevo-det-4563327.aspx?itemid=4563344
Property #9: https://www.fincaraiz.com.co/apartamento-en-venta/medellin/castellana-det-4969414.aspx
